# Templating with Jinja2

Pouring text into a pre-designed mould is very useful in several contexts. One
is is inserting dynamically generated data (e.g. retrieved from a SQL database)
into a pre-designed HTML template. Another is for creating config files for
various network devices.

Recall Python's formatted strings, or "f-strings". These provide a simple way to create text that contains data formatted as a string:

In [1]:
name = 'Ed'

f'Hello {name}'

'Hello Ed'

Jinja2 is a more powerful "templating" tool for text. It can be used to format any text file.

Jinja2 defines blocks with double braces: ``{{ ... }}`. You can apply filters
and functions to blocks - we'll come back and see some more of those shortly.
But for now this isn't very different to a Python format string:

In [3]:
from jinja2 import Template

template = Template('Hello {{ name }}!')
template.render(name='John Doe')

'Hello John Doe!'

Jinja2 templates commonly extend over multiple lines. Best practice is to keep
them in text files in a `templates` directory separate from your codebase. (In
the case of HTML templates for a website, a different team may be responsible
for editing them -- the designers rather than the back-end developers.)

First, we create a space to store our templates:

In [1]:
! mkdir templates

mkdir: cannot create directory ‘templates’: File exists


We'll start with a simple template - a basic HTML email:

In [2]:
%%writefile templates/email.html.j2

<p>Hi {{ name }}</p>

Writing templates/email.html.j2


Within Jupyter or IPython console, we can render HTML nicely like this:

In [5]:
from IPython.display import HTML

HTML('<h1>This is a big heading</h1>')

In [7]:
from pathlib import Path

template = Path('templates/email.html.j2').read_text()

HTML(
    template.render(name='Henry')
)

## Loops

Templates support loops over data items. This makes them much more powerful
than the string substitution and formatting supported by Python's strings.

Suppose you have this list of dictionaries:

In [ ]:
interfaces = [{'slotid': i, 'portid': 8000 + i, 'description': 'eth0', 'channelGroup': 'A'}
              for i in range(8)]

You could write a template to render this data as follows:

In [147]:
%%writefile templates/interfaces.cfg
{% for iface in interfaces %}
interface Ethernet{{ iface.slotid }}/{{ iface.portid }}
   description {{ iface.description }}
{% endfor %}

Overwriting templates/interfaces.cfg


In [ ]:
template = Path('templates/interfaces.cfg').read_text()
print(template.render(interfaces=interfaces))


interface Ethernet0/8000
   description eth0

interface Ethernet1/8001
   description eth0

interface Ethernet2/8002
   description eth0

interface Ethernet3/8003
   description eth0

interface Ethernet4/8004
   description eth0

interface Ethernet5/8005
   description eth0

interface Ethernet6/8006
   description eth0

interface Ethernet7/8007
   description eth0



In [153]:
text = Path('templates/interfaces.cfg').read_text()
template = Template(text)

interfaces = [{'slotid': i, 'portid': 8000 + i, 'description': 'eth0', 'channelGroup': 'A'}
              for i in range(8)]

print(template.render(interfaces=interfaces))


interface Ethernet0/8000
   description eth0

interface Ethernet1/8001
   description eth0

interface Ethernet2/8002
   description eth0

interface Ethernet3/8003
   description eth0

interface Ethernet4/8004
   description eth0

interface Ethernet5/8005
   description eth0

interface Ethernet6/8006
   description eth0

interface Ethernet7/8007
   description eth0



## Environments

Jinja2 environments let us load and use templates in a more streamlined way.
They are necessary for more advanced templating features like filters and
inheritance.

The environment is the base which Jinja uses to look up templates. We create
the environment with one (or more) loaders, which tell Jinja where to look for
templates:

In [3]:
from jinja2 import Environment, FileSystemLoader

env = Environment(loader=FileSystemLoader('templates'))  # this path can be absolute or relative

Now we can load a template as follows:

In [152]:
template = env.get_template('interfaces.cfg')

When we change the template on disk we'll need to get the template from the environment again like this:

In [ ]:
template = env.get_template('interfaces.cfg')

## Filters

Let's consider a simple example. You might have a service that is monitoring
multiple systems. Say you wanted to alter people when your machine usage got
too high.

In [12]:
import psutil

In [13]:
psutil.disk_usage('/home')

sdiskusage(total=21462233088, used=11634122752, free=9828110336, percent=54.2)

If you want to know what that is in something nicer to read, you could use the humanize library:

In [15]:
import humanize

humanize.naturalsize(21462233088)

'21.5 GB'

Say I wanted to use the nauralsize function inside my Jinja template, for any number.

In [20]:
%%writefile templates/email.html.j2

<p>Hi {{ name }},</p>
<p>The total size of the disk is {{ naturalsize(total_bytes) }}

Overwriting templates/email.html.j2


In [21]:
template = env.get_template('email.html.j2')

HTML(
    template.render(name='Ed', total_bytes=21462233088, naturalsize=humanize.naturalsize)
)

### Exercise: A nicely structured email that reports on disk size.

Update your `email.html.j2` template file so it gets the total size available,
the amount of data used, and the free space. Use the naturalsize function
inside your template to nicely render the size. Get the values from the named
tuple returned by `psutl.disk_usage`.

Update your template so that instead of multiple paramters you just pass in the
named tuple, to get the same result (*Hint* inside the template you can use the
standard dot notation to get attributes of an object).

In [22]:
%%writefile templates/email.html.j2

<p>Hi {{ name }},</p>
<p>The total size of the disk is {{ naturalsize(disk_usage.total) }}</p>
<p>The used space on the disk is {{ naturalsize(disk_usage.used) }}</p>
<p>The free space on the disk is {{ naturalsize(disk_usage.free) }}</p>

Overwriting templates/email.html.j2


In [23]:
disk_usage = psutil.disk_usage('/home')
disk_usage.free

9827622912

In [26]:
template = env.get_template('email.html.j2')

HTML(
    template.render(name='Henry', disk_usage=disk_usage, naturalsize=humanize.naturalsize)
)

Jinja also defines filters on your data. A filter is a function that transforms
the input and produces some new output. Jinja has built-in filters, but also
lets you define your own really easily:

In [28]:
env.filters['naturalsize'] = humanize.naturalsize

A filter is any function -- it should take *at least* one argument, and remaining arguments should have defaults.

In [29]:
%%writefile templates/email.html.j2

<p>Hi {{ name }},</p>
<p>The total size of the disk is {{ disk_usage.total | naturalsize }}</p>
<p>The used space on the disk is {{ disk_usage.used | naturalsize }}</p>
<p>The free space on the disk is {{ disk_usage.free | naturalsize }}</p>

Overwriting templates/email.html.j2


In [30]:
template = env.get_template('email.html.j2')

HTML(
    template.render(name='Henry', disk_usage=disk_usage)
)

In [29]:
%%writefile templates/email.html.j2

<p>Hi {{ name }},</p>
<p>The total size of the disk is {{ disk_usage.total | naturalsize }}</p>
<p>The used space on the disk is {{ disk_usage.used | naturalsize }}</p>
<p>The free space on the disk is {{ disk_usage.free | naturalsize }}</p>
<p>The current time is {{ now | timezone_convert }}

Overwriting templates/email.html.j2


In [28]:
env.filters['naturalsize'] = humanize.naturalsize

### Exercise: writing a basic filter for Jinja2

This is an example of using `pytz` to take a time-zone ignorant datetime and convert it to the time in Melbourne:

In [31]:
from datetime import datetime

In [32]:
datetime.now()

datetime.datetime(2020, 6, 18, 6, 28, 53, 856326)

In [33]:
import pytz

In [34]:
now = datetime.now()

In [35]:
pytz.utc.localize(now)

datetime.datetime(2020, 6, 18, 6, 30, 17, 869924, tzinfo=<UTC>)

In [37]:
melbourne_tz = pytz.timezone('Australia/Melbourne')  # or Australia/Perth, Sydney, Canberra, Adelaide, Darwin, Brisbane, Hobart all work

In [38]:
utc_now = pytz.utc.localize(now)

In [39]:
utc_now.astimezone(melbourne_tz)

datetime.datetime(2020, 6, 18, 16, 30, 17, 869924, tzinfo=<DstTzInfo 'Australia/Melbourne' AEST+10:00:00 STD>)

1. Write a function `timezone_convert` that given a datetime (assumed to be UTC) returns a datetime in the Melbourne timezone.
2. Add this function as a new filer to your environment - call it `timezone_convert`
3. Update your template so give a value called "now" it renders that datetime in the Melbourne timezone
4. Render the template with a time generated from `datetime.now()`

In [40]:
str(utc_now)

'2020-06-18 06:30:17.869924+00:00'

In [45]:
def timezone_convert(dt, to_tz='Australia/Melbourne', from_tz='UTC'):
    """Localize dt as the from_tz and convert to to_tz"""
    from_tz = pytz.timezone(from_tz)
    to_tz = pytz.timezone(to_tz)
    local_dt = from_tz.localize(dt)
    return local_dt.astimezone(to_tz)

In [46]:
timezone_convert(datetime.now())

datetime.datetime(2020, 6, 18, 16, 51, 53, 174910, tzinfo=<DstTzInfo 'Australia/Melbourne' AEST+10:00:00 STD>)

In [48]:
timezone_convert(datetime.now(), 'Asia/Singapore')

datetime.datetime(2020, 6, 18, 14, 52, 13, 786245, tzinfo=<DstTzInfo 'Asia/Singapore' +08+8:00:00 STD>)

In [49]:
env.filters['timezone_convert'] = timezone_convert

In [58]:
%%writefile templates/email.html.j2

<p>Hi {{ name }},</p>
<p>The total size of the disk is {{ disk_usage.total | naturalsize }}</p>
<p>The used space on the disk is {{ disk_usage.used | naturalsize }}</p>
<p>The free space on the disk is {{ disk_usage.free | naturalsize }}</p>
<p>The current time is {{ now | timezone_convert }}

Overwriting templates/email.html.j2


In [59]:
template = env.get_template('email.html.j2')

HTML(
    template.render(name='Henry', disk_usage=disk_usage, now=datetime.now())
)

Jinja has three kinds of "bracket sets"

- Blocks are surrounded by `{{ block }}`
- Flow control is surrounded by `{% for %}`
- Comments in your template are surrounded by `{#  this is a comment #}`

You can change this - when you set up the environment, the block markers are keyword arguments. Try this:

In [60]:
# help(Environment)

## Inheritance

### Prerequisite: Environments

### Exercise:

1. Create a list of dictionaries representing interfaces with the following keys:

In [ ]:
- slotid
- portid
- description
- channelGroup

2. Render your `interfaces.cfg` template, populating the data from your dictionary.

In [16]:
# Solution: see solutions/jinja2_interfaces.py

Example output:

In [94]:
!python solutions/jinja2_interfaces.py


interface Ethernet0/8000
   description eth0

interface Ethernet1/8001
   description eth0

interface Ethernet2/8002
   description eth0

interface Ethernet3/8003
   description eth0



### Example: hand-rolling YAML files via templates

This is an example of writing configuration files with custom formats. Keep in mind that good packages exist for reading and writing YAML specifically (like `pyyaml`), which would simplify writing YAML in practice.

First, read the `Data/net/rootzonedb.txt` file, which is a CSV-like file with tab separators.

In [96]:
import pandas as pd

domains = pd.read_csv('Data/net/rootzonedb.txt', sep='\t', index_col=0)

In [97]:
domains[:3]

Type                            Sponsoring Organisation
Domain                                                                   
.abogado       generic                  Top Level Domain Holdings Limited
.ac       country-code  Network Information Center (AC Domain Registry...
.academy       generic                                     Half Oaks, LLC

In [105]:
data = domains['Sponsoring Organisation']
data[:5]

Domain
.abogado                        Top Level Domain Holdings Limited
.ac             Network Information Center (AC Domain Registry...
.academy                                           Half Oaks, LLC
.accountants                                       Knob Town, LLC
.active                                   The Active Network, Inc
Name: Sponsoring Organisation, dtype: object

### Exercise:

1. Create the following YAML template to output all domains and sponsor orgs:

In [159]:
%%writefile templates/domains.yml
{% for domain, org in data.items() %}
    {{ domain }}: "{{ org | escape }}"
{% endfor %}

Overwriting templates/domains.yml


2. Write a Python script to populate the YAML from the data in `Data/net/rootzonedb.txt`.

In [165]:
%%writefile solutions/domains_and_sponsors_jinja2.py
from jinja2 import Environment, FileSystemLoader

env = Environment(loader=FileSystemLoader('templates'))
template = env.get_template('domains.yml')

import pandas as pd
domains = pd.read_csv('~/Data/net/rootzonedb.txt', sep='\t', index_col=0)
data = domains['Sponsoring Organisation']

output = template.render(data=data)

with open('new_domains.yml', mode='wt', encoding='utf8') as f:
    f.write(output)

Overwriting solutions/domains_and_sponsors_jinja2.py


In [166]:
!python solutions/domains_and_sponsors_jinja2.py

In [169]:
!head -n10 new_domains.yml


    .abogado: "Top Level Domain Holdings Limited"

    .ac: "Network Information Center (AC Domain Registry) c/o Cable and Wireless (Ascension Island)"

    .academy: "Half Oaks, LLC"

    .accountants: "Knob Town, LLC"

    .active: "The Active Network, Inc"


### Exercise:
- Use the `pyyaml` module (`import yaml`) to try loading the new YAML file you have written.

You'll notice that it's not valid YAML.

### Challenge exercise:

Fix your YAML template so it's properly quoted.

In [172]:
def yaml_escape(thing):
    return thing.replace('"', '\"')

In [178]:
%%writefile templates/domains.yml
{% for domain, org in data.items() %}
    {{ domain }}: "{{ escape(org) }}"
{% endfor %}

Overwriting templates/domains.yml


In [179]:
from jinja2 import Environment, FileSystemLoader

env = Environment(loader=FileSystemLoader('templates'))
template = env.get_template('domains.yml')

import pandas as pd
domains = pd.read_csv('~/Data/net/rootzonedb.txt', sep='\t', index_col=0)
data = domains['Sponsoring Organisation']

output = template.render(data=data, escape=yaml_escape)

with open('new_domains.yml', mode='wt', encoding='utf8') as f:
    f.write(output)